In [4]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp

In [ ]:
def euler_method(f, t0, y0, t_end, h):
    """
    Solve ODE using Euler's method for comparison
    """
    t_vals = np.arange(t0, t_end + h, h)
    y_vals = np.zeros(len(t_vals))
    y_vals[0] = y0
    
    for i in range(len(t_vals) - 1):
        y_vals[i+1] = y_vals[i] + h * f(t_vals[i], y_vals[i])
    
    return t_vals, y_vals

def modified_euler(f, t0, y0, t_end, h):
    """
    Solve ODE using Modified Euler (Heun's) method
    """
    t_vals = np.arange(t0, t_end + h, h)
    y_vals = np.zeros(len(t_vals))
    y_vals[0] = y0
    
    for i in range(len(t_vals) - 1):
        k1 = h * f(t_vals[i], y_vals[i])
        k2 = h * f(t_vals[i] + h, y_vals[i] + k1)
        y_vals[i+1] = y_vals[i] + 0.5 * (k1 + k2)
    
    return t_vals, y_vals

def midpoint_method(f, t0, y0, t_end, h):
    """
    Solve ODE using Midpoint method
    """
    t_vals = np.arange(t0, t_end + h, h)
    y_vals = np.zeros(len(t_vals))
    y_vals[0] = y0
    
    for i in range(len(t_vals) - 1):
        k1 = h * f(t_vals[i], y_vals[i])
        k2 = h * f(t_vals[i] + h/2, y_vals[i] + k1/2)
        y_vals[i+1] = y_vals[i] + k2
    
    return t_vals, y_vals

In [ ]:
class Coupled_Oscillators:
    def __init__(self, N, m = 1.0, k_springs = 1.0, left_wall_k=0.0, right_wall_k=0.0):

        """Inicializa sistema com N massas e N-1 molas (mais molas opcionais nas paredes).
        Args:
        N (int): number of masses
        m (float ou array): massa de cada oscilador; se um único valor, aplica-se a todas as massas
        k_springs (float ou array): N-1 constantes das molas between; se um único valor, aplica-se a todas as molas
        left_wall_k (float): constante da mola na parede esquerda
        right_wall_k (float): constante da mola na parede direita"""
          
        self.N=N
        self.M = self._build_mass_matrix(m)
        self.K = self._build_K_matrix(k_springs, left_wall_k, right_wall_k)

    def _build_mass_matrix(self, m):

        if isinstance(m, (int, float)):  
            # se um único valor, aplica-se a todas as massas
            masses = np.full(self.N, m)
        elif isinstance(m, (list, np.ndarray)) and len(m) == self.N:
            masses = np.array(m)
        else:
            raise ValueError("m deve ser um float ou um array de N entradas")
        
        return np.diag(masses)

    def _build_K_matrix(self, k_springs, left_wall_k, right_wall_k):

            N = self.N
            if isinstance(k_springs, (int, float)):
                k_array = np.full(N - 1, k_springs)
            elif isinstance(k_springs, (list, np.ndarray)) and len(k_springs) == N - 1:
                k_array = np.array(k_springs)
            else:
                raise ValueError("k_springs deve ser float ou array de N-1 entradas")

            # Inicializa matriz K
            K = np.zeros((N, N))

            # Preenche matriz com ligações entre massas
            for i in range(N - 1):
                K[i, i] += k_array[i]
                K[i + 1, i + 1] += k_array[i]
                K[i, i + 1] -= k_array[i]
                K[i + 1, i] -= k_array[i]

            # Ligações às paredes
            K[0, 0] += left_wall_k
            K[N - 1, N - 1] += right_wall_k

            return K


    def solve_coupled_system_linear(self, x0=None, v0=None, t_max=50, num_points=1000):
        
        return sol.t, sol.y, t_eval, u_euler
